Pseudocode for Permutation Core Butson-Type Complex Hadamard Matrices:

* Select dimension $n$ for matrix
* Generate multiples of $\Phi_n(x)$ with sum of coefficient magnitudes equal to $n$
* Normalize so that they're all positive coefficients by multiplying by $x^{n/2}$ for the negative ones

In [7]:
# type: ignore

import itertools

# -----------------------------
# 1. Generate positive-coefficient L1-bounded rows
# -----------------------------
def generate_L1_vectors_positive(d, n):
    """
    Yield tuples (c0,...,cd-1) with:
      - all ci >= 0
      - sum(ci) == n
      - c0 >= 1 (dephasing)
    """
    coeffs = [0]*d

    def backtrack(i, remaining):
        if i == d:
            if remaining == 0 and coeffs[0] >= 1:
                yield tuple(coeffs)
            return
        start = 1 if i == 0 else 0
        for val in range(start, remaining+1):
            coeffs[i] = val
            yield from backtrack(i+1, remaining - val)
            coeffs[i] = 0

    yield from backtrack(0, n)

# -----------------------------
# 2. Orthogonality check using polynomials on the fly
# -----------------------------
def rows_orthogonal(row1, row2, d):
    R = ZZ['x']
    x = R.gen()
    Phi = cyclotomic_polynomial(d)
    # build polynomials on the fly
    p1 = sum(c*x^i for i, c in enumerate(row1))
    p2 = sum(c*x^i for i, c in enumerate(row2))
    return (p1 * p2) % Phi == 0

# -----------------------------
# 3. Recursive construction of permutation-core CHMs
# -----------------------------
def build_permutation_core_CHMs(core_coeffs, n):
    """
    core_coeffs: list of integers (first entry >= 1)
    d: size of the CHM
    Returns: list of matrices (each matrix is a list of rows, each row is a list)
    """
    first_row = [1]*n
    fixed_first = core_coeffs[0]
    tail = core_coeffs[1:]
    results = []

    def backtrack(existing_rows):
        if len(existing_rows) == n:
            results.append([row[:] for row in existing_rows])
            return
        for perm in itertools.permutations(tail):
            candidate_row = [fixed_first] + list(perm)
            if all(rows_orthogonal(candidate_row, row, n) for row in existing_rows):
                existing_rows.append(candidate_row)
                backtrack(existing_rows)
                existing_rows.pop()

    backtrack([first_row])
    return results

# -----------------------------
# 4. Example usage
# -----------------------------
d = 4   # CHM size / degree
n = 3   # sum of coefficients for core row

for coeffs in generate_L1_vectors_positive(d, n):
    chms = build_permutation_core_CHMs(list(coeffs), n)
    if chms:
        print(f"Core coefficients: {coeffs}")
        for mat in chms:
            for row in mat:
                print(row)
            print("---")


Core coefficients: (1, 0, 1, 1)
[1, 1, 1]
[1, 0, 1, 1]
[1, 1, 1, 0]
---
[1, 1, 1]
[1, 0, 1, 1]
[1, 1, 1, 0]
---
[1, 1, 1]
[1, 0, 1, 1]
[1, 1, 1, 0]
---
[1, 1, 1]
[1, 0, 1, 1]
[1, 1, 1, 0]
---
[1, 1, 1]
[1, 1, 0, 1]
[1, 1, 1, 0]
---
[1, 1, 1]
[1, 1, 0, 1]
[1, 1, 1, 0]
---
[1, 1, 1]
[1, 1, 1, 0]
[1, 0, 1, 1]
---
[1, 1, 1]
[1, 1, 1, 0]
[1, 0, 1, 1]
---
[1, 1, 1]
[1, 1, 1, 0]
[1, 1, 0, 1]
---
[1, 1, 1]
[1, 1, 1, 0]
[1, 1, 1, 0]
---
[1, 1, 1]
[1, 1, 1, 0]
[1, 1, 0, 1]
---
[1, 1, 1]
[1, 1, 1, 0]
[1, 1, 1, 0]
---
[1, 1, 1]
[1, 1, 0, 1]
[1, 1, 1, 0]
---
[1, 1, 1]
[1, 1, 0, 1]
[1, 1, 1, 0]
---
[1, 1, 1]
[1, 1, 1, 0]
[1, 0, 1, 1]
---
[1, 1, 1]
[1, 1, 1, 0]
[1, 0, 1, 1]
---
[1, 1, 1]
[1, 1, 1, 0]
[1, 1, 0, 1]
---
[1, 1, 1]
[1, 1, 1, 0]
[1, 1, 1, 0]
---
[1, 1, 1]
[1, 1, 1, 0]
[1, 1, 0, 1]
---
[1, 1, 1]
[1, 1, 1, 0]
[1, 1, 1, 0]
---
Core coefficients: (1, 1, 0, 1)
[1, 1, 1]
[1, 1, 0, 1]
[1, 1, 1, 0]
---
[1, 1, 1]
[1, 1, 0, 1]
[1, 1, 1, 0]
---
[1, 1, 1]
[1, 1, 1, 0]
[1, 1, 0, 1]
---
[1, 1, 1]
[1, 1,

Definition: A cycle is a full sum of roots of unity, possibly rotated by a scalar. Geometrically, it represents a regular polygon on the unit circle.

Let $l=p_1^{a_1}\cdots p_k^{a_k}$, and assume that $\lambda_i\in\mathbb{Z}_l$ satisfy $\lambda_1+\cdots+\lambda_N=0$.

If $k\leq 2$, then $\sum p_i^{a_i}$ is a sum of cycles with coefficients in $\mathbb{N}$.

If $k\geq 3$, then $\sum p_i^{a_i}$ is a sum of cycles with coefficients in $\mathbb{Z}$, but not necessarily in $\mathbb{n}$.

Here, we'll focus our search on products of 2 prime powers, i.e. 6. That way, we'll have sums of cycles.

The gist of what I want to write here is going to be the following:

Function for generating polynomials that sum to 0 in our root of unity.
* For roots with 1 prime divisor, this is a fairly trivial exercise because it usually doesn't work.
* For roots with 2 prime divisors, they can be broken down into rotated full sums of prime roots of unity.
* For roots with 3 or more prime divisors, they can be broken down into POSITIVE OR NEGATIVE full sums of prime roots of unity.

In the meantime, focus on writing the program for the roots with 2 prime divisors. It's mostly a partition problem for that one.

Once I've got the polynomials from the first function, I then want to run them through multiple ways of generating Butson-type CHMs.
* For the permutation core Butson-type CHMs, I just want to see if I can take said rows and use an individual row to generate an entire matrix.
* For the general Butson-type CHMs, I'd need to use a more elaborate function that takes multiple candidate rows (along with their permutations) and determines if they work, before moving on to other candidates. Not sure how to handle the indexing and sorting for this one in particular.
* It probably *does* make sense to try to write things with a similar workflow to the way the AI wrote it, i.e. dedicated functions for generating acceptable rows, permuting said rows (might be a library thing), testing orthogonality between a candidate row and a list of other (fixed-position) rows, and constructing a matrix.
* The main thing that seems to pose a challenge is going to be the iteration process. I'm not sure how to go about this, but I'm sure there's a way.